In [ ]:
# Importing necessary libraries for randomness, numerical computations, and PyTorch functionalities
import random
import numpy as np
import torch

# Setting a fixed seed to ensure reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Configuring PyTorch backend to make computations deterministic for CUDA
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## **CNN for Text Classification**


### **Load dataset**

In [ ]:
!git clone https://github.com/congnghia0609/ntc-scv.git

In [ ]:
!unzip ./ntc-scv/data/data_test.zip -d ./data
!unzip ./ntc-scv/data/data_train.zip -d ./data
!rm -rf ./ntc-scv

In [ ]:
import os
import pandas as pd


def load_data_from_path(folder_path):
	examples = []
	for label in os.listdir(folder_path):
		full_path = os.path.join(folder_path, label)
		for file_name in os.listdir(full_path):
			file_path = os.path.join(full_path, file_name)
			with open(file_path, "r", encoding = "utf-8") as f:
				lines = f.readlines()
			sentence = " ".join(lines)
			if label == "neg":
				label = 0
			if label == "pos":
				label = 1
			data = {
				'sentence': sentence,
				'label': label
			}
			examples.append(data)
	return pd.DataFrame(examples)


folder_paths = {
	'train': './data/data_train/train',
	'valid': './data/data_train/test',
	'test': './data/data_test/test'
}

train_df = load_data_from_path(folder_paths['train'])
valid_df = load_data_from_path(folder_paths['valid'])
test_df = load_data_from_path(folder_paths['test'])


### **Preprocessing**

In [ ]:


from langid.langid import LanguageIdentifier, model


def identify_vn(df):
	identifier = LanguageIdentifier.from_modelstring(model, norm_probs = True)
	not_vi_idx = set()
	THRESHOLD = 0.9
	for idx, row in df.iterrows():
		score = identifier.classify(row["sentence"])
		if score[0] != "vi" or (score[0] == "vi" and score[1] <= THRESHOLD):
			not_vi_idx.add(idx)
	vi_df = df[~df.index.isin(not_vi_idx)]
	not_vi_df = df[df.index.isin(not_vi_idx)]
	return vi_df, not_vi_df


train_df_vi, train_df_other = identify_vn(train_df)

import re
import string


def preprocess_text(text):
	# remove URLs https://www.
	url_pattern = re.compile(r'https?://\s+\wwww\.\s+')
	text = url_pattern.sub(r" ", text)

	# remove HTML Tags: <>
	html_pattern = re.compile(r'<[^<>]+>')
	text = html_pattern.sub(" ", text)

	# remove puncs and digits
	replace_chars = list(string.punctuation + string.digits)
	for char in replace_chars:
		text = text.replace(char, " ")

	# remove emoji
	emoji_pattern = re.compile("["
	                           u"\U0001F600-\U0001F64F"  # emoticons
	                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
	                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
	                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
	                           u"\U0001F1F2-\U0001F1F4"  # Macau flag
	                           u"\U0001F1E6-\U0001F1FF"  # flags
	                           u"\U0001F600-\U0001F64F"
	                           u"\U00002702-\U000027B0"
	                           u"\U000024C2-\U0001F251"
	                           u"\U0001f926-\U0001f937"
	                           u"\U0001F1F2"
	                           u"\U0001F1F4"
	                           u"\U0001F620"
	                           u"\u200d"
	                           u"\u2640-\u2642"
	                           "]+", flags = re.UNICODE)
	text = emoji_pattern.sub(r" ", text)

	# normalize whitespace
	text = " ".join(text.split())

	# lowercasing
	text = text.lower()
	return text


train_df_vi['preprocess_sentence'] = [preprocess_text(row['sentence']) for index, row in train_df_vi.iterrows()]
valid_df['preprocess_sentence'] = [preprocess_text(row['sentence']) for index, row in valid_df.iterrows()]
test_df['preprocess_sentence'] = [preprocess_text(row['sentence']) for index, row in test_df.iterrows()]

train_df_vi

### **Representation**

In [ ]:

def yield_tokens(sentences, tokenizer):
	for sentence in sentences:
		yield tokenizer(sentence)



In [ ]:

# word-based tokenizer
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer("basic_english")



In [ ]:
# build vocabulary
from torchtext.vocab import build_vocab_from_iterator

vocab_size = 10000
vocabulary = build_vocab_from_iterator(
	yield_tokens(train_df_vi['preprocess_sentence'], tokenizer),
	max_tokens = vocab_size,
	specials = ["<pad>", "<unk>"]
)
vocabulary.set_default_index(vocabulary["<unk>"])

vocabulary["<pad>"]


In [ ]:

from torchtext.data.functional import to_map_style_dataset


def prepare_dataset(df):
	# create iterator for dataset: (sentence, label)
	for index, row in df.iterrows():
		sentence = row['preprocess_sentence']
		encoded_sentence = vocabulary(tokenizer(sentence))
		label = row['label']
		yield encoded_sentence, label


train_dataset = prepare_dataset(train_df_vi)
train_dataset = to_map_style_dataset(train_dataset)

valid_dataset = prepare_dataset(valid_df)
valid_dataset = to_map_style_dataset(valid_dataset)

### **Dataloader**

In [ ]:
###**Dataloader**

import torch
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def collate_batch(batch):
	# create inputs, offsets, labels for batch
	encoded_sentences, labels = [], []
	for encoded_sentence, label in batch:
		labels.append(label)
		encoded_sentence = torch.tensor(encoded_sentence, dtype = torch.int64)
		encoded_sentences.append(encoded_sentence)

	labels = torch.tensor(labels, dtype = torch.int64)
	encoded_sentences = pad_sequence(
		encoded_sentences,
		padding_value = vocabulary["<pad>"]
	)

	return encoded_sentences, labels


from torch.utils.data import DataLoader

batch_size = 128
train_dataloader = DataLoader(
	train_dataset,
	batch_size = batch_size,
	shuffle = True,
	collate_fn = collate_batch
)
valid_dataloader = DataLoader(
	valid_dataset,
	batch_size = batch_size,
	shuffle = False,
	collate_fn = collate_batch
)

next(iter(train_dataloader))

encoded_sentences, labels = next(iter(train_dataloader))

encoded_sentences.shape

### **Model**

In [ ]:
###**Model**

import torch.nn as nn
from model import TextCNN

vocab_size = len(vocabulary)
embedding_dim = 100

model = TextCNN(
	vocab_size = vocab_size,
	embedding_dim = embedding_dim,
	kernel_sizes = [3, 4, 5],
	num_filters = 100,
	num_classes = 2
)

vocab_size



In [ ]:
predictions = model(encoded_sentences)
predictions


In [ ]:
predictions.shape


### **Loss & Optimizer**

In [ ]:

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

loss = criterion(predictions, labels)
loss


### **Trainer**

In [ ]:
import time


def train(model, optimizer, criterion, train_dataloader, device, epoch = 0, log_interval = 50):
	model.train()
	total_acc, total_count = 0, 0
	losses = []
	start_time = time.time()

	for idx, (inputs, labels) in enumerate(train_dataloader):
		inputs = inputs.to(device)
		labels = labels.to(device)

		# zero grad
		optimizer.zero_grad()

		# predictions
		predictions = model(inputs)

		# compute loss
		loss = criterion(predictions, labels)
		losses.append(loss.item())

		# backward
		loss.backward()
		optimizer.step()
		total_acc += (predictions.argmax(1) == labels).sum().item()
		total_count += labels.size(0)
		if idx % log_interval == 0 and idx > 0:
			elapsed = time.time() - start_time
			print(
				"| epoch {:3d} | {:5d}/{:5d} batches "
				"| accuracy {:8.3f}".format(
					epoch, idx, len(train_dataloader), total_acc / total_count
				)
			)
			total_acc, total_count = 0, 0
			start_time = time.time()

	epoch_acc = total_acc / total_count
	epoch_loss = sum(losses) / len(losses)
	return epoch_acc, epoch_loss


model.to(device)

epoch_acc, epoch_loss = train(model, optimizer, criterion, train_dataloader, device)

epoch_acc, epoch_loss

In [ ]:

def evaluate(model, criterion, valid_dataloader, device):
	model.eval()
	total_acc, total_count = 0, 0
	losses = []

	with torch.no_grad():
		for idx, (inputs, labels) in enumerate(valid_dataloader):
			inputs = inputs.to(device)
			labels = labels.to(device)
			# predictions
			predictions = model(inputs)

			# compute loss
			loss = criterion(predictions, labels)
			losses.append(loss.item())

			total_acc += (predictions.argmax(1) == labels).sum().item()
			total_count += labels.size(0)

	epoch_acc = total_acc / total_count
	epoch_loss = sum(losses) / len(losses)
	return epoch_acc, epoch_loss


eval_acc, eval_loss = evaluate(model, criterion, valid_dataloader, device)

eval_acc, eval_loss

### **Training**

In [ ]:


import torch.nn.functional as F

num_class = 2
vocab_size = len(vocabulary)
embedding_dim = 300
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextCNN(
	vocab_size = vocab_size,
	embedding_dim = embedding_dim,
	kernel_sizes = [3, 4, 5],
	num_filters = 100,
	num_classes = 2
)
model.to(device)

criterion = torch.nn.CrossEntropyLoss()

learning_rate = 2e-4
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

num_epochs = 10
save_model = './model'

train_accs, train_losses = [], []
eval_accs, eval_losses = [], []
best_loss_eval = 100

for epoch in range(1, num_epochs + 1):
	epoch_start_time = time.time()
	# Training
	train_acc, train_loss = train(model, optimizer, criterion, train_dataloader, device, epoch)
	train_accs.append(train_acc)
	train_losses.append(train_loss)

	# Evaluation
	eval_acc, eval_loss = evaluate(model, criterion, valid_dataloader, device)
	eval_accs.append(eval_acc)
	eval_losses.append(eval_loss)

	# Save best model
	if eval_loss < best_loss_eval:
		print('Save model at ./model/text_cnn_model.pt')
		torch.save(model.state_dict(), save_model + '/text_cnn_model.pt')

	# Print loss, acc end epoch
	print("-" * 59)
	print(
		"| End of epoch {:3d} | Time: {:5.2f}s | Train Accuracy {:8.3f} | Train Loss {:8.3f} "
		"| Valid Accuracy {:8.3f} | Valid Loss {:8.3f} ".format(
			epoch, time.time() - epoch_start_time, train_acc, train_loss, eval_acc, eval_loss
		)
	)
	print("-" * 59)

	# Load best model
	model.load_state_dict(torch.load(save_model + '/text_cnn_model.pt'))
	model.eval()

import matplotlib.pyplot as plt


def plot_result(num_epochs, train_accs, eval_accs, train_losses, eval_losses):
	epochs = list(range(num_epochs))
	fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 6))
	axs[0].plot(epochs, train_accs, label = "Training")
	axs[0].plot(epochs, eval_accs, label = "Evaluation")
	axs[1].plot(epochs, train_losses, label = "Training")
	axs[1].plot(epochs, eval_losses, label = "Evaluation")
	axs[0].set_xlabel("Epochs")
	axs[1].set_xlabel("Epochs")
	axs[0].set_ylabel("Accuracy")
	axs[1].set_ylabel("Loss")
	plt.legend()


plot_result(num_epochs, train_accs, eval_accs, train_losses, eval_losses)

### **Evaluation & Prediction**

In [ ]:

test_dataset = prepare_dataset(test_df)
test_dataset = to_map_style_dataset(test_dataset)

test_dataloader = DataLoader(
	test_dataset,
	batch_size = batch_size,
	shuffle = False,
	collate_fn = collate_batch
)

test_acc, test_loss = evaluate(model, criterion, test_dataloader, device)
test_acc, test_loss

### **Inference**

In [ ]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer("basic_english")


def load_model(model_path, vocab_size = 10000, embedding_dim = 300, num_classes = 2):
	model = TextCNN(
		vocab_size = vocab_size,
		embedding_dim = embedding_dim,
		kernel_sizes = [3, 4, 5],
		num_filters = 100,
		num_classes = num_classes
	)
	model.load_state_dict(torch.load(model_path, weights_only = True))
	model.eval()
	return model


def inference(sentence, vocabulary, model):
	sentence = preprocess_text(sentence)
	encoded_sentence = vocabulary(tokenizer(sentence))
	encoded_sentence = torch.tensor(encoded_sentence)
	encoded_sentence = torch.unsqueeze(encoded_sentence, 1)

	with torch.no_grad():
		predictions = model(encoded_sentence)
	preds = nn.Softmax(dim = 1)(predictions)
	p_max, yhat = torch.max(preds.data, 1)
	return round(p_max.item(), 2) * 100, yhat.item()


In [ ]:
torch.save(model.state_dict(), save_model + '/text_cnn_model.pt')


In [ ]:

model = load_model('./model/text_cnn_model.pt')
inference(test_df['preprocess_sentence'][5999], vocabulary, model)

test_df

In [ ]:
inference('Đồ ăn rất ngon và rẻ', vocabulary, model)

test_df['preprocess_sentence'][5999]